# Convolutional neural network

## Load X,y data from NPZ

Using the function added to *mylib.py* file, it's now easy to grab data and X/y vectors ready to be used for model training and tuning

In [1]:
# Run content of mylib.py file
%run mylib.py

# Load data from NPZ file
#data=loadNpz()
(data, X, y)=loadXy(verbose=False)

https://www.tutorialspoint.com/how-can-keras-be-used-to-extract-features-from-only-one-layer-of-the-model-using-python

In [4]:
import numpy as np


IMAGE_SIZE=data['IMAGE_SIZE']

IMAGE_COLOR_SPACE_DIM=data['IMAGE_COLOR_SPACE_DIM']

NB_LABELS=data['IMAGE_NB_LABELS']

FEATURE_SIZE=data['FEATURE_SIZE']

DATASET_NAME=data['DATASET_NAME']


KERAS_INPUT_SHAPE=(IMAGE_SIZE[0], IMAGE_SIZE[1], IMAGE_COLOR_SPACE_DIM)


X_input=data['trainX']['data']

X_input=np.float32(X_input)



In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# Some constant 
BATCH_SIZE=32

# Create image generators for each dataset
image_batches=dict()

for name in DATASET_NAME:
    print("\nLoading {} set".format(name))
    image_batches[name]=ImageDataGenerator(rescale=1./255).flow_from_directory(
        os.path.join('data',name),
        batch_size=BATCH_SIZE,
        target_size=IMAGE_SIZE,
        shuffle=False
    )
    


Loading train set
Found 280 images belonging to 6 classes.

Loading test set
Found 50 images belonging to 6 classes.

Loading valid set
Found 139 images belonging to 6 classes.


In [32]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow.keras as keras
from tensorflow.keras import activations

# Build placeholder shape from our global constant
PLACEHOLDER_SHAPE=(None, IMAGE_SIZE[0], IMAGE_SIZE[1], IMAGE_COLOR_SPACE_DIM)

FEATURE_SIZE=2048

# Create graph
img_graph = tf.Graph()

with img_graph.as_default():
    # Convolutional Network
    model = keras.Sequential()
    model.add(keras.layers.Conv2D(filters=32, kernel_size=5, strides=2,
                                  activation='relu', input_shape=KERAS_INPUT_SHAPE))
    model.add(keras.layers.MaxPool2D(pool_size=3))
    model.add(keras.layers.Conv2D(filters=32, kernel_size=3, strides=1,
                                  activation='relu'))
    model.add(keras.layers.MaxPool2D(pool_size=2))
    model.add(keras.layers.Flatten(name='test'))
    model.add(keras.layers.Dense(units=2048, activation='relu', name='test2'))
    model.add(keras.layers.Dense(units=6, activation='softmax', name='test3'))

    


    feature_extractor = keras.Model(
       inputs=model.inputs,
       outputs=model.get_layer(name="test2").output,
    )
    
    # Create input placeholder
    input_imgs = tf.placeholder(dtype=tf.float32, shape=PLACEHOLDER_SHAPE)

    # A node with the features
    imgs_features = feature_extractor(X_input)

    # Collect initializers
    init_op = tf.group([
        tf.global_variables_initializer(), tf.tables_initializer()
    ])

img_graph.finalize() # Good practice: make the graph "read-only"

In [33]:
import numpy as np

data_dict_new=dict()

# Create a session
sess = tf.Session(graph=img_graph)

# Initialize it
sess.run(init_op)

DATASET_NAME=['test']

# For each set
for name in DATASET_NAME:
    print('Processing {} dataset, total number of batches: {}'.format(name, len(image_batches[name])))
    
    # Initialize new dict for this new dataset name
    data_dict_new[name]=dict()
    
    # Add filenames to data_dict
    data_dict_new[name]['filenames']=image_batches[name].filenames
    
    # Add label id of each image
    data_dict_new[name]['labels']=image_batches[name].labels

    
    # Initialize empty array for data and feature var
    # Those var will be used to accumulate data between for loops
    # They are initialized to empty array with the correct shape
    data=np.empty((0, IMAGE_SIZE[0], IMAGE_SIZE[1], IMAGE_COLOR_SPACE_DIM))
    features=np.empty((0, FEATURE_SIZE))
 
    # For each batch of each set
    for batch in range(0, len(image_batches[name])):
        print('  batch number {}'.format(batch))
        
        # Append new data batch to previous data batches
        data=np.append(data, image_batches[name][batch][0], axis=0)

        # Get high-level features from data
        features=np.append(features, sess.run(imgs_features, feed_dict={input_imgs: image_batches[name][batch][0]}), axis=0)


        # print(features.shape)
        # train_features.extend(features)
        
    # Store data in global data_dict
    data_dict_new[name]['data']=data
    
    # Store features in global dict
    data_dict_new[name]['features']=features



Processing test dataset, total number of batches: 2
  batch number 0
  batch number 1


In [34]:
data_dict_new['test']['features'].shape



(840, 2048)

In [27]:

# Convolutional Network
    model = keras.Sequential()
    model.add(keras.layers.Conv2D(filters=32, kernel_size=5, strides=2,
                                  activation='relu', input_shape=KERAS_INPUT_SHAPE))
    model.add(keras.layers.MaxPool2D(pool_size=3))
    model.add(keras.layers.Conv2D(filters=32, kernel_size=3, strides=1,
                                  activation='relu'))
    model.add(keras.layers.MaxPool2D(pool_size=2))
    model.add(keras.layers.Flatten(name='test'))
    model.add(keras.layers.Dense(units=2048, activation='relu', name='test2'))
    model.add(keras.layers.Dense(units=10, activation='softmax', name='test3'))


    
    model.summary()
    

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_14 (Conv2D)           (None, 148, 148, 32)      2432      
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 49, 49, 32)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 47, 47, 32)        9248      
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 23, 23, 32)        0         
_________________________________________________________________
test (Flatten)               (None, 16928)             0         
_________________________________________________________________
test2 (Dense)                (None, 2048)              34670592  
_________________________________________________________________
test3 (Dense)                (None, 10)                20490     
Total para